# Random Graph Experimentation - Comparing RobustPC variants vs PC algorithm

We hypothesize that RobustPC in the context of **faithful** and/or **m-strong-faithful** graphs will perform better than the naive PC algorithm. The baseline CPDAG we want to compare to is the one using a ParentOracle, which is uniformly consistent for all graphs.

We have the following comparisons that leverage the momentary conditional independence condition (MCI), which leverages a fixed conditioning set in addition to that suggested via PC:

    - definite parents: Using a parent oracle
    - definite children: Using a children oracle
    - definite parents + definite children: Using a parent + children oracle
    - definite Markov blanket: Using the markov blanket oracle
    - estimated parents: using "some" procedure to estimate the parents
    - estimated parents + children: using "some" procedure to estimate the parents+children
    - estimated markov blanket: estimating the markov blanket
    
References
----------
[1] https://www.jmlr.org/papers/volume9/pellet08a/pellet08a.pdf

In [2]:
%load_ext lab_black

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
from pathlib import Path
from pprint import pprint
import json

import numpy as np
import bnlearn as bn
import networkx as nx
import pandas as pd

import causal_networkx
from causal_networkx.ci import (
    g_square_discrete,
    fisherz,
    g_square_binary,
    PartialCorrelation,
    Oracle,
    ParentOracle,
)
from causal_networkx.discovery import PC, RobustPC
from causal_networkx.io import read_dot, load_from_networkx

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
np.random.seed(12345)

# Set up to load the data

The data should have been previously generated via the manm_cs package for additive noise models. We will load the data in with specific parameters.

In [8]:
data_dir = Path("/Volumes/Extreme Pro/structure_learning_manm_sim")

In [15]:
ci_est = "parcorr"
ci_estimator = PartialCorrelation()

In [23]:
for idx in range(310, 320):
    data_fname = data_dir / f"graph_{idx}.csv"
    graph_fname = data_dir / f"graph_{idx}.gml"
    meta_fname = data_dir / f"graph_{idx}.json"

    # load the actual data
    df = pd.read_csv(data_fname, index_col=0)
    nx_graph = nx.read_gml(graph_fname)
    true_graph = load_from_networkx(nx_graph)
    with open(meta_fname, "r") as fin:
        meta_dict = json.load(fin)

    # sub-sample data
    df = df.sample(2000)

    # run the PC algorithm with Oracle
    oracle = Oracle(true_graph)
    pc_oracle = PC(ci_estimator=oracle.ci_test)
    pc_oracle.fit(df)
    
    oracle_graph = pc_oracle.graph_
    fname = data_dir / f'graph_oracle_{idx}.gml'
    oracle_graph.save(fname, format="networkx-gml")

    # run the PC algorithm with partial correlation
    pc_alg = PC(ci_estimator=parcorr.test)
    pc_alg.fit(df)
    
    pc_graph = pc_oracle.graph_
    fname = data_dir / f'graph_pcalg_{idx}.gml'
    pc_graph.save(fname, format="networkx-gml")
    
    # ci_estimator=ci_estimator,
    # max_cond_set_size=max_cond_set_size,
    # alpha=alpha,
    # max_combinations=None,
    # max_conds_x=max_conds_x,
    # max_conds_y=max_conds_y,
    # mci_alpha=mci_alpha,
    # partial_knowledge=partial_knowledge,
    # size_inclusive=size_inclusive,
    # use_children=False,
    # skip_first_stage=True,
    # only_mci=True,
    
    # run the RobustPC algorithm with partial correlation with ParentOracle
    partial_knowledge = ParentOracle(true_graph)
    robustpc = RobustPC(ci_estimator=ci_estimator,
                        partial_knowledge=partial_knowledge,
                        skip_first_stage=True,
                        use_children=False)
    
    robustpc.fit(df)
    graph = robustpc.graph_
    fname = data_dir / f'graph_robust_defparents_{ci_est}_{idx}.gml'
    graph.save(fname, format="networkx-gml")
    
    # run the RobustPC algorithm with partial correlation with ParentChildrenOracle
    partial_knowledge = ParentOracle(true_graph)
    robustpc = RobustPC(ci_estimator=ci_estimator,
                       partial_knowledge=partial_knowledge,
                       skip_first_stage=True,
                       use_children=True,
                       )
    robustpc.fit(df)
    graph = robustpc.graph_
    fname = data_dir / f'graph_robust_defparentschildren_{ci_est}_{idx}.gml'
    graph.save(fname, format="networkx-gml")
    
    # run the RobustPC algorithm with partial correlation with ParentChildrenOracle
    # partial_knowledge = ParentOracle(true_graph)
    # robustpc = RobustPC(ci_estimator=parcorr.test,
    #                    partial_knowledge=partial_knowledge,
    #                    skip_first_stage=True,)
        
    # true_graph.draw()
    # print(true_graph)
    # print(meta_dict)
    break

TypeError: 'PartialCorrelation' object is not callable

In [19]:
print(df.shape)
df = df.sample(2000)
print(df.shape)
display(df.head())
print(true_graph.nodes)

(2000, 8)
(2000, 8)


,0,1,3,2,4,5,6,7
35745,0.003403,0.033700,0.203421,-0.076177,0.249018,0.123770,-0.134373,0.177194
40994,-0.112126,-0.120526,0.181267,-0.009901,0.055066,-0.036829,-0.099851,-0.107016
6769,0.232365,0.080657,-0.295970,0.099740,0.010127,0.073952,-0.094826,-0.209772
2676,-0.042552,0.021746,-0.037615,-0.027329,-0.075735,0.104392,0.128094,-0.102721
28080,0.124283,-0.275674,-0.236886,0.141696,0.084311,0.114855,0.422301,-0.198861


['0', '1', '2', '3', '4', '5', '6', '7']


In [30]:
print(true_graph.edges)

[('0', '3'), ('0', '4'), ('0', '6'), ('1', '2'), ('1', '6'), ('2', '7'), ('3', '4'), ('3', '5'), ('3', '6'), ('4', '5'), ('5', '6'), ('5', '7'), ('6', '7')]
True


## Try with Simulations

In [7]:
seed = 12345
rng = np.random.RandomState(seed)
X = rng.randn(300, 1)
X1 = rng.randn(300, 1)
Y = np.concatenate((X, X), axis=1) + 0.5 * rng.randn(300, 2)
Z = Y + 0.5 * rng.randn(300, 2)

In [ ]:
ci_estimator = PartialCorrelation()
_, pvalue = ci_estimator.test(X, X1)

In [ ]:
import numpy as np
import scipy.stats
x = np.array([0, 1, 2, 3, 5, 8, 13])
y = np.array([1.2, 1.4, 1.6, 1.7, 2.0, 4.1, 6.6])

r, p = scipy.stats.pearsonr(x, y)